# TP1 - Parte B: Classificação de Atividades Humanas 
### Tópicos de Ciência dos Dados - MECD (2022/2023)

### Autores

- Duarte Meneses - 2019216949
- Patricia Costa - 2019213995


In [3]:
import csv
import os
import numpy as np
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d
from sklearn import datasets
from sklearn.decomposition import PCA
import pandas as pd

In [219]:
columns_labels = [ 'Device ID', 'accelerometer x', 'accelerometer y', 'accelerometer z',
    'gyroscope x', 'gyroscope y', 'gyroscope z', 'magnetometer x', 'magnetometer y', 'magnetometer z',
    'Timestamp', 'Activity Label']

activities = np.array(['Stand', 'Sit', 'Sit and Talk', 'Walk', 'Walk and Talk', 'Climb Stair (up/down)',
    'Climb Stair (up/down) and talk', 'Stand -> Sit', 'Sit -> Stand', 'Stand -> Sit and talk', 'Sit -> Stand and talk',
    'Stand -> walk', 'Walk -> stand', 'Stand -> climb stairs (up/down)(and talk)', 'Climb stairs (up/down) -> walk',
    'Climb stairs (up/down) and talk -> walk and talk'])

sensors = ['Acc', 'Gyr', 'Mag']

## 4.5 e 4.6 da Parte A

## Parte B


### Dataset

In [38]:
# import some data to play with
iris = datasets.load_iris()

data = iris.data[:, :2]  # we only take the first two features.
target = iris.target

### 1.1. Data Splitting

### 1.1.1. Train-Test (TT) e Train-Validation-Test data split

In [51]:
from sklearn.model_selection import train_test_split
#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

def train_test(datav, targetv, train_size, shuffle = False):
    trainTest = []

    x_train, x_test, y_train, y_test = train_test_split(datav, targetv, train_size=train_size, shuffle=shuffle)

    trainTest.append({"TRAIN": [x_train, y_train], "TEST": [x_test, y_test]})

    return trainTest

def train_validation_test(data, target, train_size, validation_size, shuffle = False):
    trainValidation = []

    x_train, x_rem, y_train, y_rem = train_test_split(data, target, train_size=train_size, shuffle=shuffle)

    test_size = 1 - train_size - validation_size
    x_valid, x_test, y_valid, y_test = train_test_split(x_rem, y_rem, test_size = test_size, shuffle=shuffle)

    trainValidation.append({"TRAIN": [x_train, y_train], "TEST": [x_test, y_test], "Validation": [x_valid, y_valid]})
    
    return trainValidation



### 1.1.2. K-fold data split

In [52]:
from sklearn.model_selection import KFold
#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html

def k_fold(x, y, n_splits, shuffle = False, random_state = None):
    kf = KFold(n_splits=n_splits, random_state=random_state, shuffle=shuffle)
    kfold = []
    for train_index, test_index in kf.split(x):
        x_train, x_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
        print("TRAIN:", [x_train, y_train])
        print("TEST:", [x_test, y_test])
        kfold.append({"TRAIN": [x_train, y_train], "TEST": [x_test, y_test]})
    
    return kfold

In [53]:
#Testes
X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]])
y = np.array([1, 2, 3, 4])

print("--Train Test--\n")
trainTest = train_test(X, y, 0.7)
display(trainTest)

print("\n--Train Validation Test--\n")
trainValidation = train_validation_test(X, y, 0.4, 0.3)
display(trainValidation)

print("\n--K Fold--\n")
kFold = k_fold(X, y, 4)
display(kFold)

--Train Test--



[{'TRAIN': [array([[1, 2],
          [3, 4]]),
   array([1, 2])],
  'TEST': [array([[1, 2],
          [3, 4]]),
   array([3, 4])]}]


--Train Validation Test--



[{'TRAIN': [array([[1, 2]]), array([1])],
  'TEST': [array([[3, 4]]), array([4])],
  'Validation': [array([[3, 4],
          [1, 2]]),
   array([2, 3])]}]


--K Fold--

TRAIN: [array([[3, 4],
       [1, 2],
       [3, 4]]), array([2, 3, 4])]
TEST: [array([[1, 2]]), array([1])]
TRAIN: [array([[1, 2],
       [1, 2],
       [3, 4]]), array([1, 3, 4])]
TEST: [array([[3, 4]]), array([2])]
TRAIN: [array([[1, 2],
       [3, 4],
       [3, 4]]), array([1, 2, 4])]
TEST: [array([[1, 2]]), array([3])]
TRAIN: [array([[1, 2],
       [3, 4],
       [1, 2]]), array([1, 2, 3])]
TEST: [array([[3, 4]]), array([4])]


[{'TRAIN': [array([[3, 4],
          [1, 2],
          [3, 4]]),
   array([2, 3, 4])],
  'TEST': [array([[1, 2]]), array([1])]},
 {'TRAIN': [array([[1, 2],
          [1, 2],
          [3, 4]]),
   array([1, 3, 4])],
  'TEST': [array([[3, 4]]), array([2])]},
 {'TRAIN': [array([[1, 2],
          [3, 4],
          [3, 4]]),
   array([1, 2, 4])],
  'TEST': [array([[1, 2]]), array([3])]},
 {'TRAIN': [array([[1, 2],
          [3, 4],
          [1, 2]]),
   array([1, 2, 3])],
  'TEST': [array([[3, 4]]), array([4])]}]

### 1.2. Métricas de exactidação

In [41]:
#Teste
true = [2, 0, 2, 2, 0, 1]
pred = [0, 0, 2, 2, 0, 2]

-> Explicação do parâmetro average

**'binary'**:
Only report results for the class specified by pos_label. This is applicable only if targets (y_{true,pred}) are binary.

**'micro'**:
Calculate metrics globally by counting the total true positives, false negatives and false positives.

**'macro'**:
Calculate metrics for each label, and find their unweighted mean. This does not take label imbalance into account.

**'weighted'**:
Calculate metrics for each label, and find their average weighted by support (the number of true instances for each label). This alters ‘macro’ to account for label imbalance; it can result in an F-score that is not between precision and recall.

**'samples'**:
Calculate metrics for each instance, and find their average (only meaningful for multilabel classification where this differs from accuracy_score).

In [46]:
from sklearn.metrics import confusion_matrix
#https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html#sklearn.metrics.confusion_matrix
from sklearn.metrics import recall_score
#https://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html#sklearn.metrics.recall_score
from sklearn.metrics import precision_score
#https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html#sklearn.metrics.precision_score
from sklearn.metrics import f1_score
#https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score

def metrics(true, pred):
    cm = confusion_matrix(true, pred)
    rs = recall_score(true, pred, average='weighted', zero_division=0)
    ps = precision_score(true, pred, average='weighted', zero_division=0)
    f1 = f1_score(true, pred, average='weighted', zero_division=0)

    print('Confusion Matrix:\n', cm)
    print('\nRecall: ', rs)
    print('\nPrecision: ', ps)
    print('\nF1-score: ', f1)
    print('-'*50)

    return [cm, rs, ps, f1]

def visualize(cm, rs, ps, f1):
    cmFrame = pd.DataFrame(cm).style.set_caption("Confusion Matrix")
    display(cmFrame)
    metricsFrame = pd.DataFrame([rs, ps, f1], columns=['Metrics']).rename(index={0: 'Recall', 1: 'Precision', 2: 'F1-score'})
    display(metricsFrame)
    

cm, rs, ps, f1 = metrics(true, pred)
visualize(cm, rs, ps, f1)



Confusion Matrix:
 [[2 0 0]
 [0 0 1]
 [1 0 2]]

Recall:  0.6666666666666666

Precision:  0.5555555555555555

F1-score:  0.6
--------------------------------------------------


,0,1,2
0,2,0,0
1,0,0,1
2,1,0,2


,Metrics
Recall,0.666667
Precision,0.555556
F1-score,0.600000
